# Features 

En este notebook se irán generando distintos features que luego deberán ser probados.

In [34]:
import pandas as pd
import numpy as np
import re
import string
#from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df['id'] = df['id'].astype(np.uint16)
df['target'] = df['target'].astype(np.uint8)
#df = df.drop(labels='location', axis=1)
df = df.fillna("")

## Features a partir de la keyword 

#### One Hot Encoding de las Keywords 

In [35]:
#ohe_keywords = OneHotEncoder(sparse=False, dtype=np.uint8, handle_unknown="ignore")

In [ ]:
#ohe_keywords

#### Count Encoding de las keywords 

In [4]:
keywords = df["keyword"].value_counts()

In [14]:
df["keyword_count"] = df["keyword"].transform(lambda x: keywords[x])

Versión normalizada:

In [15]:
df["keyword_count_norm"] = df["keyword_count"]/keywords.max()

#### Mean Encoding de las keywords 

In [17]:
keywords_target = df.groupby(["keyword", "target"]).count()["id"]

In [21]:
df["keyword_mean"] = df.agg(lambda x: keywords_target[x["keyword"]][x["target"]], axis=1)

## Features a partir del tweet 

Feature que indica si en el tweet hay algún número o no:

In [30]:
def hay_nros(x):
    x = x.split()
    for i in x:
        i = i.replace(',','')
        try:
            float(i)
            return 1
        except ValueError:
            continue
    return 0 

df["hay_nros"] = df["text"].transform(hay_nros)

Feature que indica la longitud del tweet:

In [32]:
df["long"] = df["text"].transform(lambda x: len(x))

#### Feature Hashing de 101 columnas para el tweet con modelo BOW 

In [ ]:
# Función para limpiar el texto de los mensajes.
def clean_text(text):
    # Se convierte el texto a minúsculas.
    text = text.lower()
    # Se quitan los '#'.
    text = re.sub('#', '', text)
    # Se quitan los números.
    text = re.sub('\w*\d\w*', '', text)
    # Se quitan los saltos de línea.
    text = re.sub('\n', ' ', text)
    # Se eliminan las referencias a usuarios '@user'.
    text = re.sub('@\S*', '', text)
    # Se quitan vínculos URL.
    text = re.sub('https{0,1}:\/\/\S*', ' ', text)
    # Se simplifican múltiples espacios a uno solo.
    text = re.sub('(\ ){2,7}', ' ',text)
    # Se quitan los signos de puntuación.
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    return text